![alt text](..\img\header.jpg)

# Exercise 3: MODFLOW 6
The purpose of this exercise is to use MODFLOW 6 and the DISV package to simulate example problem 1 using Flopy.

## Part I. Setup Notebook

In [ ]:
%matplotlib inline
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

## Part II. Create, Run, and Post-Process MODFLOW 6 Model

In [ ]:
# model info
model_name = 'ex03'
model_ws = './ex03-completed'

# grid properties
nlay = 3
nrow = 21
ncol = 20
ncpl = nrow * ncol
delr = 500.
delc = 500.
top = 400.
botm = [220., 200., 0.]

# mapping array from row and column to cell number
mapcell = np.arange(nrow * ncol, dtype=np.int).reshape((nrow, ncol))

# hydraulic properties
hk0 = 50.
vk0 = 10.
hk1 = 0.01
vk1 = 0.01
hk2 = 200.
vk2 = 20

# boundary conditions
welspd = [[(3 - 1, mapcell[11 - 1, 10 - 1]), -1.5e5]]
rech = 0.005
rivspd = [[(0, mapcell[ir, ncol-1]), 320., 1.e5, 318.] for ir in range(nrow)]

In [ ]:
xv = np.linspace(0, 500. * ncol, ncol+1)
yv = np.linspace(500 * nrow, 0, nrow + 1)
xv, yv = np.meshgrid(xv, yv)
xv = xv.ravel()
yv = yv.ravel()

def get_vlist(i, j, nrow, ncol):
    v1 = i * (ncol + 1) + j
    v2 = v1 + 1
    v3 = v2 + ncol + 1
    v4 = v3 - 1
    return [v1, v2, v3, v4]

iverts = []
for i in range(nrow):
    for j in range(ncol):
        iverts.append(get_vlist(i, j, nrow, ncol))

nvert = xv.shape[0]
verts = np.hstack((xv.reshape(nvert, 1), yv.reshape(nvert, 1)))

cellxy = np.empty((nvert, 2))
for icpl in range(ncpl):
    iv = iverts[icpl]
    cellxy[icpl, 0] = (xv[iv[0]] + xv[iv[1]]) / 2.
    cellxy[icpl, 1] = (yv[iv[1]] + yv[iv[2]]) / 2.

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
flopy.plot.plotutil.plot_cvfd(verts, iverts, facecolor='none', ax=ax)
ax.plot(xv, yv, 'bo')
ax.plot(cellxy[:, 0], cellxy[:, 1], 'ro')
ax.set_xlim(xv.min(), xv.max())
ax.set_ylim(yv.min(), yv.max())

In [ ]:
# need to create cell2d, which is [[icpl, xc, yc, nv, iv1, iv2, iv3, iv4]]
cell2d = [[icpl, cellxy[icpl, 0], cellxy[icpl, 1], 4, *iverts[icpl]] for icpl in range(ncpl)]
vertices = [[ivert, verts[ivert, 0], verts[ivert, 1]] for ivert in range(nvert)]

In [ ]:
# create simulation
sim = flopy.mf6.MFSimulation(sim_name=model_name, version='mf6', exe_name=config.mf6exe, 
                             sim_ws=model_ws)

# create tdis package
tdis_rc = None #[(1000.0, 1, 1.0)]
tdis = flopy.mf6.ModflowTdis(sim)

# create gwf model
gwf = flopy.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file='{}.nam'.format(model_name))
gwf.name_file.save_flows = True

# create iterative model solution and register the gwf model with it
ims = flopy.mf6.ModflowIms(sim)

# disv
disv = flopy.mf6.ModflowGwfdisv(gwf, nlay=nlay, ncpl=ncpl, 
                                top=top, botm=botm, 
                                nvert=nvert, vertices=vertices, 
                                cell2d=cell2d)

# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, pname='ic', strt=320.)

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True,
                              icelltype=[1,0,0], 
                              k=[50.0, 0.01, 200.0],
                              k33=[10., 0.01, 20.])

# wel
wel = flopy.mf6.ModflowGwfwel(gwf, print_input=True,
                              stress_period_data=welspd)

# rch
aux = [np.ones((nrow, ncol), dtype=np.int)*6]
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=0.005, 
                               auxiliary=[('iface',)], 
                               aux={0: [6]})
# riv
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data=rivspd)

# output control
oc = flopy.mf6.ModflowGwfoc(gwf, pname='oc', budget_filerecord='{}.cbc'.format(model_name),
                            head_filerecord='{}.hds'.format(model_name),
                            headprintrecord=[('COLUMNS', 10, 'WIDTH', 15,
                                              'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])

sim.write_simulation()
sim.run_simulation()

In [ ]:
# Load binary grid file
fname = os.path.join(model_ws, model_name + '.disv.grb')
grd = flopy.utils.MfGrdFile(fname, verbose=False)
iverts, verts = grd.get_verts()
vertc = grd.get_centroids()
xc = vertc[:, 0]
yc = vertc[:, 1]

# load the heads and budget files
fname = os.path.join(model_ws, model_name + '.hds')
hdobj = flopy.utils.HeadFile(fname)
head = hdobj.get_data()

fname = os.path.join(model_ws, model_name + '.cbc')
cbb = flopy.utils.CellBudgetFile(fname, precision='double')
spdis = cbb.get_data(text='DATA-SPDIS')[0]
qx = spdis['qx'].reshape(nlay, nrow, ncol)
qy = spdis['qy'].reshape(nlay, nrow, ncol)
qz = spdis['qz'].reshape(nlay, nrow, ncol)

In [ ]:
# plot the results for each layer
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))
vmin = head.min()
vmax = head.max()
levels = np.linspace(vmin, vmax, 20)
gwf.modelgrid.set_coord_info(angrot=10.)
xcentergrid = gwf.modelgrid.xcellcenters
ycentergrid = gwf.modelgrid.ycellcenters

for ilay in range(nlay):
    ax = axes[ilay]
    ax.set_title('Layer {}'.format(ilay + 1))
    ax.set_aspect('equal')
    pmv = flopy.plot.PlotMapView(model=gwf, layer=ilay, ax=ax)
    pmv.plot_grid()
    v = pmv.plot_array(head[ilay, 0, :], cmap='jet', ax=ax)
    cs = pmv.contour_array(head[ilay, 0, :], colors='white', levels=levels, ax=ax)
    plt.clabel(cs, fmt='%.1f', colors='white', fontsize=11)
    hmin = head[ilay, 0, :].min()
    hmax = head[ilay, 0, :].max()
    t = ax.set_title('Model Layer {}; hmin={:6.2f}, hmax={:6.2f}'.format(ilay + 1, hmin, hmax))
    ax.quiver(xcentergrid, ycentergrid, qx[ilay], qy[ilay])
cb = fig.colorbar(v, ax=axes, shrink=0.5)

## Part III. Create and Run MODPATH-7 Model

In [ ]:
# particles
partlocs = [(mapcell[i, 0]) for i in range(nrow)]
particledata = flopy.modpath.ParticleData(partlocs, structured=False)
pg = flopy.modpath.ParticleGroup(particledata=particledata)

In [ ]:
mp = flopy.modpath.Modpath7(modelname=model_name+'_mp', model_ws=model_ws, flowmodel=gwf, exe_name=config.mpexe)
mpbas = flopy.modpath.Modpath7Bas(mp)
mpsim = flopy.modpath.Modpath7Sim(mp, particlegroups=pg)
mp.write_input()
mp.run_model()

In [ ]:
# load the pathline results
fname = os.path.join(model_ws, model_name + '_mp.mppth')
p = flopy.utils.PathlineFile(fname).get_alldata()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
mm = flopy.plot.PlotMapView(model=gwf)
mm.plot_grid(lw=0.5)
mm.plot_pathline(p, layer='all', colors='k', label='Particle Paths')
mm.ax.legend();